# Ingestion de la carpeta "movie_language"

In [0]:
dbutils.widgets.text("p_environment", "")
v_env = dbutils.widgets.get("p_environment")
v_env

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer los archivos CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
movie_languages_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("languageId", IntegerType(), True),
    StructField("languageRoleId", IntegerType(), True),
])

In [0]:
movies_languages_df = spark.read.json(f'{bronze_folder_path}/movie_language/', schema=movie_languages_schema, multiLine=True)

In [0]:
movies_languages_df.count()

In [0]:
movies_languages_df.printSchema()

### Paso 2 - Renombrar columnas y adicionar columnas "ingestion_date" y "env"

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
movies_language_renamed_df = movies_languages_df \
    .withColumnRenamed("movieId", "movie_id") \
    .withColumnRenamed("languageId", "language_id") \
    .transform(add_ingestion_date) \
    .withColumn('env',lit(v_env))

In [0]:
display(movies_language_renamed_df)

## Paso 3 Eliminar columna "languageRoleId"

In [0]:
from pyspark.sql.functions import col

In [0]:
movies_language_final_df = movies_language_renamed_df.drop(col('languageRoleId'))

### Paso 4 - Escribir datos en el DataLake en formato parquet

In [0]:
# movies_language_final_df.write.mode('overwrite').parquet(f'{silver_folder_path}/movies_languages')
movies_language_final_df.write.mode('overwrite').format("parquet").saveAsTable("movie_silver.movies_languages")

In [0]:
%python
display(dbutils.fs.ls("/mnt/moviehistory/silver/movies_languages"))

In [0]:
df = spark.read.parquet('/mnt/moviehistory/silver/movies_languages')
display(df)

In [0]:
dbutils.notebook.exit("Sucess")